# <font color="#114b98">      <div align="center">   Classez des images à l'aide d'algorithmes de Deep Learning
    
    

L'association locale de protection des animaux souhaite disposer d'une API capable de reconnaître la race d'un chien à partir de sa photo. En tant que bénévoles, nous nous proposons de créer un algorithme de reconnaissance d'image pour accélérer le processus d'indexation de l'association.

Cependant, les bénévoles de l'association n'ont pas encore eu l'opportunité de rassembler toutes les images des chiens résidant dans leur refuge, car celles-ci sont dispersées sur différents disques durs. Pour résoudre ce problème, nous prévoyons d'utiliser la base de données "Stanford Dogs Dataset".

Le code final à déployer se trouve dans ce notebook, et le modèle final choisi est Xception .


In [2]:
# python librairies
import pandas as pd
import numpy as np
import cv2
from keras.models import load_model

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 1. Chargement du modèle entrainé

Nous allons charger le modèle qui a été entraîné sur l'ensemble des données. Ce modèle est basé sur l'architecture pré-entraînée d'Xception, à laquelle la couche dense supérieure a été retirée et toutes les autres couches ont été gelées.


In [ ]:
# Load model
cnn_tf = load_model('/content/drive/MyDrive//Colab Notebooks/Ingenieur IA/Mission_5 Classer_image_deep Learning/Xception_model.h5')
cnn_tf.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 10, 10, 2048)      20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 120)               61560     
                                                                 
Total params: 21,972,128
Trainable params: 1,110,648
Non-trainable params: 20,861,480
____________________________________

In [4]:
# Load breeds
classes_ = pd.read_csv("/content/drive/MyDrive//Colab Notebooks/Ingenieur IA/Mission_5 Classer_image_deep Learning/classes.csv")
classes_.head()

,breed_name
0,Chihuahua
1,Japanese_spaniel
2,Maltese_dog
3,Pekinese
4,Shih


### 2. Api de prédiction

Nous allons élaborer un script qui permettra de prédire la race d'un chien en utilisant une image de ce dernier comme entrée. Pour ce faire, nous ferons appel à la bibliothèque Gradio, qui nous aidera à importer les images et à appliquer notre modèle pré-entraîné pour obtenir les cinq meilleures prédictions de la race du chien présent sur l'image importée.

In [ ]:
!pip install gradio

In [ ]:

nb_classes = classes_.shape[1]
classes = classes_.breed_name.values
# Define the full prediction function
def predict_pred(image):
    # Convert to RGB
    im_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize image
    dim = (299, 299)
    img = cv2.resize(image, dim)
    # Equalization
    r_image, g_image, b_image = cv2.split(img)
    r_image_eq = cv2.equalizeHist(r_image)
    g_image_eq = cv2.equalizeHist(g_image)
    b_image_eq = cv2.equalizeHist(b_image)
    image_eq = cv2.merge((r_image_eq, g_image_eq, b_image_eq))

    # Convert modified img to array

    img_ar = image_eq.reshape((-1, 299, 299, 3))

    # Predictions
    prediction = cnn_tf.predict(img_ar)[0]
    breeds=[]
    # for classe in encoder.classes_:
    #  breeds.append(classe.split('-')[1])
    #prediction
    return {classes[i]: float(prediction[i]) for i in range(len(classes))}

# Construct the interface
import gradio as gr

gr.Interface(fn=predict_pred,
             inputs=gr.Image(),
             outputs=gr.Label(num_top_classes=5)).launch(share=True)




Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://4bca3970cd8fe1886b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
